In [1]:
from pyautocad import Autocad, APoint
from pyautocad import aDouble
import pandas as pd
import ast

In [2]:
# cad is the varialbe carrying the Autocad object
acad = Autocad(create_if_not_exists = True) 


## Functions

In [4]:
# Prints the file name of the cad file connected.
def cad_file_name(cad_object):
    print("The cad file name connected is: ", cad_object.doc.name)

# **********************************************************************************************

# Prints the current set units of the Autocad file
def current_units(cad_object):
    units_dict = dict(zip(range(1,6), 
                  ["Scientific", "Decimal", "Engineering", "Architectural", "Fractionsl"]))

    print("The current set unit is: ", units_dict[acad.doc.GetVariable("LUNITS")])

# **********************************************************************************************

# Sets the units to desired units.
def set_units(cad_object, unit_name):
    units_dict = dict(zip(["Scientific", "Decimal", "Engineering", "Architectural", "Fractionsl"], range(1,6)))
    if unit_name in units_dict.keys():
        cad_object.doc.SetVariable("LUNITS", units_dict[unit_name])
    current_units(cad_object)

# **********************************************************************************************

# Function to draw a line as per given start and end coordinates.
def draw_line(start_coord: tuple, end_coord: tuple):
    start_point = APoint(start_coord[0], start_coord[1])
    end_point = APoint(end_coord[0], end_coord[1])
    line = acad.model.AddLine(start_point, end_point)
    print("Line drawn successfully!")

# **********************************************************************************************

# Function to DELETE THE BLOCKS DEFINED
def delete_block(block_to_delete_name: str):
    
    print(f"\nAttempting to delete block '{block_to_delete_name}' and all its instances.")
    
    # Step 1: Delete all instances of the block from the model space
    for obj in acad.model:
        if obj.ObjectName == 'AcDbBlockReference' and obj.Name == block_to_delete_name:
            try:
                obj.Delete()
                print(f"Deleted an instance of '{block_to_delete_name}'.")
            except Exception as e:
                print(f"Could not delete instance: {e}")
    
    # Step 2: Delete the block definition itself
    try:
        # Get the block definition from the Blocks collection
        block_def_to_delete = acad.doc.Blocks.Item(block_to_delete_name)
        # Delete it
        block_def_to_delete.Delete()
        print(f"Successfully deleted block definition '{block_to_delete_name}'.")
    except Exception as e:
        print(f"Could not delete block definition '{block_to_delete_name}'. Error: {e}")

# **********************************************************************************************

# 
def insert_block(insertion_point: tuple, block_name: str):
    block_insertion_point = APoint(insertion_point[0]*1000, insertion_point[1]*1000)
    acad.model.InsertBlock(
        block_insertion_point, # Insertion point for this instance
        block_name,              # Name of the block to insert
        1, 1, 1,                 # X, Y, Z scale factors
        0                        # Rotation angle (in radians)
    )
    print("Insertion successful")

# insert_block(grid_df.loc[1,'B'], "Footing_F1")

# **********************************************************************************************

# Function to DRAW HORIZONTAL GRID LINES
def draw_h_grid(start_coord: tuple, end_coord: tuple, ext ):
    start_point = APoint(start_coord[0]*1000 - ext, start_coord[1]*1000)
    end_point = APoint(end_coord[0]*1000 + ext, end_coord[1]*1000)
    line = acad.model.AddLine(start_point, end_point)
    
    line.Layer = 'STR_center'                             # set the layer name for the grid lines
    print("Line drawn successfully!")

# **********************************************************************************************

# Function to DRAW HORIZONTAL GRID LINES
def draw_v_grid(start_coord: tuple, end_coord: tuple, ext ):
    start_point = APoint(start_coord[0]*1000, 1000*start_coord[1] - ext)
    end_point = APoint(1000*end_coord[0], 1000*end_coord[1] + ext)
    line = acad.model.AddLine(start_point, end_point)
    
    line.Layer = 'STR_center'                            # set the layer name for the grid lines
    
    print("Line drawn successfully!")
            

In [5]:
cad_file_name(acad)

The cad file name connected is:  str_template.dwg


In [6]:
acad.ActiveDocument.Layers

### Importing excel file containing the grid coordinate data

In [8]:
grid_df = pd.read_excel("coordinates.xlsx", index_col = 0)

grid_df = grid_df.map(ast.literal_eval)   
# Transforms the cell values to the pandas data types. 
# for example (3.0, 0.0) is initially a string and using ast converts it into 
# pandas data type which is a tuple
print(grid_df)

            A           B           C
1  (0.0, 0.0)  (3.0, 0.0)  (6.0, 0.0)
2  (0.0, 4.0)  (3.0, 4.0)  (6.0, 4.0)
3  (0.0, 8.0)  (3.0, 8.0)  (6.0, 8.0)


## Grid Line Drawer Function

In [10]:

def draw_h_grid(start_coord: tuple, end_coord: tuple, ext ):
    start_point = APoint(start_coord[0]*1000 - ext, start_coord[1]*1000)
    end_point = APoint(end_coord[0]*1000 + ext, end_coord[1]*1000)
    line = acad.model.AddLine(start_point, end_point)
    
    line.Layer = 'STR_center'                             # set the layer name for the grid lines
    print("Line drawn successfully!")

def draw_v_grid(start_coord: tuple, end_coord: tuple, ext ):
    start_point = APoint(start_coord[0]*1000, 1000*start_coord[1] - ext)
    end_point = APoint(1000*end_coord[0], 1000*end_coord[1] + ext)
    line = acad.model.AddLine(start_point, end_point)
    
    line.Layer = 'STR_center'                            # set the layer name for the grid lines
    
    print("Line drawn successfully!")

In [11]:
# Drawing grid lines
# Horizontal Grid Lines
start_id = grid_df.columns[0]
end_id = grid_df.columns[-1]
for y_id in grid_df.index:
    start_coord = grid_df.loc[y_id, start_id]
    end_coord = grid_df.loc[y_id, end_id]
    draw_h_grid(start_coord, end_coord, 3000)  # enter the extension value in mm

# Vertical Grid Lines
start_id = grid_df.index[0]
end_id = grid_df.index[-1]
for x_id in grid_df.columns:
    start_coord = grid_df.loc[start_id, x_id]
    end_coord = grid_df.loc[end_id, x_id]
    draw_v_grid(start_coord, end_coord, 3000)  # enter the extension value in mm

Line drawn successfully!
Line drawn successfully!
Line drawn successfully!
Line drawn successfully!
Line drawn successfully!
Line drawn successfully!


## Function to draw Block for Footing

In [13]:
def footing_block(insertion_point: tuple, width: int, height:int, block_name:str, layer_name: str):
    
    center_point = APoint(insertion_point[0], insertion_point[1])
    footing_block_def = acad.doc.Blocks.Add(center_point, block_name)
    
    # 2. Calculate all four vertices
    p1 = APoint(center_point.x-width/2, center_point.y-height/2) # Bottom-left
    p2 = APoint(p1.x + width, p1.y) # Bottom-right
    p3 = APoint(p1.x + width, p1.y + height) # Top-right
    p4 = APoint(p1.x, p1.y + height) # Top-left
    
    # 3. Create a flat list of coordinates for the polyline
    # The format must be (x1, y1, x2, y2, x3, y3, x4, y4)
    # vertices = [p1.x, p1.y,0, p2.x, p2.y, 0, p3.x, p3.y, 0, p4.x, p4.y, 0]
    vertices = [p1.x, p1.y, p2.x, p2.y, p3.x, p3.y, p4.x, p4.y]
    
    # 4. Create the polyline object
    rectangle = footing_block_def.AddLightWeightPolyline(aDouble(vertices))

    # 5. Setting the layer to the rectangle object
    rectangle.Layer = layer_name
    
    # 6. Close the polyline to complete the rectangle
    rectangle.Closed = True
       
    print(f'Block {block_name} created successfully')

### Footing Type DataFrame from Excel

In [15]:
# DataFrame which contains the footing type data that is read from an excel file
footing_type_df = pd.read_excel('footing_data.xlsx',sheet_name= 'footing_dimensions', index_col = 0)
print(footing_type_df)

     LENGTH  WIDTH  DEPTH
FF1    1500   1500    350
FF2    1800   1800    350
FF3    2000   2000    400


In [16]:
# Block creator for all the different footing types
for type in footing_type_df.index:
    width = footing_type_df.loc[type]['LENGTH']
    height = footing_type_df.loc[type]['WIDTH']
    block_name = type
    layer_name = 'STR_FOOTING'
    footing_block((0,0), width, height, block_name, layer_name)

Block FF1 created successfully
Block FF2 created successfully
Block FF3 created successfully


### Footing Layout DataFrame from Excel

In [18]:
# DataFrame which contains the location of different footing types at the coordinats required
footing_layout_df = pd.read_excel('footing_data.xlsx', sheet_name = 'footing_layout', index_col = 0)
print(footing_layout_df)

     A    B    C
1  FF1  FF1  FF2
2  FF3  FF2  FF1
3  FF1  FF2  FF3


In [19]:
# Inserting the footing types at the required positions
x_id = grid_df.columns
y_id = grid_df.index
for xid in x_id:
    for yid in y_id:
        insert_block(grid_df.loc[yid][xid], footing_layout_df.loc[yid][xid])

Insertion successful
Insertion successful
Insertion successful
Insertion successful
Insertion successful
Insertion successful
Insertion successful
Insertion successful
Insertion successful


In [37]:
for type in footing_type_df.index:
    delete_block(type)


Attempting to delete block 'FF1' and all its instances.
Successfully deleted block definition 'FF1'.

Attempting to delete block 'FF2' and all its instances.
Successfully deleted block definition 'FF2'.

Attempting to delete block 'FF3' and all its instances.
Successfully deleted block definition 'FF3'.
